In [2]:
!pip install polars --no-index --find-links=file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg

Looking in links: file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
Processing /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg/polars-0.20.16-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import polars as pl
#import duckdb as dd
#from tqdm import tqdm
import matplotlib.pyplot as plt
#import cv2
#from pydicom import dcmread
import warnings
#from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import pickle
#import gc
import ctypes
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow import keras

E0000 00:00:1727504102.192805      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
D0928 06:15:02.201438501      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D0928 06:15:02.201453182      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D0928 06:15:02.201456497      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D0928 06:15:02.201458931      13 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D0928 06:15:02.201461303      13 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defa

In [4]:
"""print('Running on TPU ', TPU.master())
except ValueError:
    print('Running on GPU')
    TPU = None

if TPU:
    IS_TPU = True
    tf.config.experimental_connect_to_cluster(TPU)"""

# detect TPUs
tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

print("Number of accelerators: ", tpu_strategy.num_replicas_in_sync)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1727504119.638172      13 service.cc:145] XLA service 0x5b738e7fe240 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727504119.638249      13 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1727504119.638253      13 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1727504119.638257      13 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1727504119.638261      13 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1727504119.638264      13 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1727504119.638267      13 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1727504119.638270      13 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1727504119.638273      13 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [5]:
def read_and_parse_dicom_files_tensorflow_train(full_file_path):
    #tf.config.run_functions_eagerly(True)
    raw_image = tf.io.read_file(full_file_path)
    sp = tf.strings.split(tf.gather(tf.strings.split(full_file_path, 'images/'), 1), '/')
    N = tf.size(sp)
    LEN = tf.strings.length(tf.gather(sp, 0))+tf.strings.length(tf.gather(sp, 2))
    
    # Add missing file metadata to avoid warnnigs flooding
    if   LEN==12: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x92\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==13: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x92\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==14: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x94\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==15: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x94\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==16: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x96\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==17: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x96\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==18: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x98\x00\x00\x00\x02\x00\x01\x00')
    
    image = tfio.image.decode_dicom_image(raw_image, scale='auto', dtype=tf.float32)
    m, M=tf.math.reduce_min(image), tf.math.reduce_max(image)
    image = (tf.image.grayscale_to_rgb(image)-m)/(M-m)
    image = tf.image.resize(image, (128,128))
    sqzd_image = tf.squeeze(image)
    #shaped_image = sqzd_image.set_shape(sqzd_image.get_shape())
    return sqzd_image

def preprocessing(img_path):
    train_img = read_and_parse_dicom_files_tensorflow_train(img_path)
    train_img = tf.reshape(train_img, shape=(128, 128, 3))
    return train_img

def load_dataset_tensorflow_train(image_path, labels):
    image = preprocessing(image_path)
    return {"images": tf.cast(image, tf.float32), "labels": tf.cast(labels, tf.float32)}

def dict_to_tuple(inputs):
    return inputs["images"], inputs["labels"]

In [6]:
condition_for_training = 'left_neural_foraminal_narrowing'
vertebrae_position = 'l1_l2'

metadata_file_path = '/kaggle/input/left-neural-foraminal-narrowing-metadata/{0}_{1}_feature_metadata.csv'.format(condition_for_training, vertebrae_position)
metadata_df = pl.read_csv(metadata_file_path, low_memory=True)

x_train, x_test_val = train_test_split(metadata_df, test_size=0.4, random_state=42)
x_test, x_valid = train_test_split(x_test_val, test_size=0.2, random_state=42)

print("Training data shape : {0}".format(x_train.shape))
print("Test data shape : {0}".format(x_test.shape))
print("Validation data shape : {0}".format(x_valid.shape))

Training data shape : (88249, 3)
Test data shape : (47067, 3)
Validation data shape : (11767, 3)


In [ ]:
train_image_filenames = pl.Series(x_train.select(pl.col('full_img_path'))).to_list()
train_image_labels = pl.Series(x_train.select(pl.col('encoded_severity'))).to_list()

In [ ]:
#trial_img = read_and_parse_dicom_files_tensorflow_train(train_image_filenames[0])
prep_img = preprocessing(train_image_filenames[0])

In [ ]:

#trial_img.get_shape()
"""shaped_img = trial_img.set_shape(trial_img.get_shape())
type(shaped_img)"""

"""t = tf.constant([[[1, 1, 1], [2, 2, 2]], [[3, 3, 3], [4, 4, 4]]])
tf.rank(t)"""

tf.rank(prep_img)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_image_filenames, train_image_labels))

In [ ]:
train_ds = train_dataset.map(load_dataset_tensorflow_train, num_parallel_calls=tf.data.AUTOTUNE)

In [9]:
def generate_tf_datasets(p_train_df, p_test_df, p_valid_df, p_BATCH_SIZE_PER_REPLICA):

    BATCH_SIZE = p_BATCH_SIZE_PER_REPLICA * tpu_strategy.num_replicas_in_sync
    
    train_image_filenames = pl.Series(p_train_df.select(pl.col('full_img_path'))).to_list()
    train_image_labels = pl.Series(p_train_df.select(pl.col('encoded_severity'))).to_list()

    test_image_filenames = pl.Series(p_test_df.select(pl.col('full_img_path'))).to_list()
    test_image_labels = pl.Series(p_test_df.select(pl.col('encoded_severity'))).to_list()

    valid_image_filenames = pl.Series(p_valid_df.select(pl.col('full_img_path'))).to_list()
    valid_image_labels = pl.Series(p_valid_df.select(pl.col('encoded_severity'))).to_list()
    
    
    train_dataset = tf.data.Dataset.from_tensor_slices((train_image_filenames, train_image_labels))
    test_dataset = tf.data.Dataset.from_tensor_slices((test_image_filenames, test_image_labels))
    valid_dataset = tf.data.Dataset.from_tensor_slices((valid_image_filenames, valid_image_labels))
    
    train_ds = train_dataset.map(load_dataset_tensorflow_train, num_parallel_calls=tf.data.AUTOTUNE)
    train_ds = train_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
    train_ds = train_ds.batch(batch_size=BATCH_SIZE, drop_remainder=True)
    train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

    test_ds = test_dataset.map(load_dataset_tensorflow_train, num_parallel_calls=tf.data.AUTOTUNE)
    test_ds = test_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
    test_ds = test_ds.batch(batch_size=BATCH_SIZE, drop_remainder=True)
    test_ds = test_ds.prefetch(tf.data.AUTOTUNE)

    valid_ds = valid_dataset.map(load_dataset_tensorflow_train, num_parallel_calls=tf.data.AUTOTUNE)
    valid_ds = valid_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
    valid_ds = valid_ds.batch(batch_size=BATCH_SIZE, drop_remainder=True)
    valid_ds = valid_ds.prefetch(tf.data.AUTOTUNE)
    
    return train_ds, test_ds, valid_ds

In [10]:
train_ds, test_ds, valid_ds = generate_tf_datasets(p_train_df=x_train, p_test_df=x_test, p_valid_df=x_valid
                                                   , p_BATCH_SIZE_PER_REPLICA = 16)

In [11]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers

with tpu_strategy.scope():
    
    rsna_input = layers.Input(shape=(128,128,3), name="rsna_input")

    conv_base = EfficientNetB0(include_top=False, weights="imagenet", input_tensor=rsna_input)
    conv_base.trainable = False

    max_pool_layer_0 = layers.MaxPooling2D(name="max_pool_0", pool_size=(2, 2), strides=(1, 1), padding="same")(conv_base.output)
    max_pool_layer_0 = layers.BatchNormalization()(max_pool_layer_0)

    conv2d_1 = layers.Conv2D(filters=100, kernel_size=4, strides=1, padding="same", activation="relu")(max_pool_layer_0)
    max_pool_layer_1 = layers.MaxPooling2D(name="max_pool_1", pool_size=(2, 2), strides=(1, 1), padding="same")(conv2d_1)
    max_pool_layer_1 = layers.BatchNormalization()(max_pool_layer_1)

    flattened_layer = layers.Flatten()(max_pool_layer_1)

    hidden_layer1 = layers.Dense(200, activation="selu", kernel_initializer=keras.initializers.LecunNormal(seed=None))(flattened_layer)
    hidden_layer1 = layers.BatchNormalization()(hidden_layer1)
    hidden_layer2 = layers.Dense(100, activation="selu", kernel_initializer=keras.initializers.LecunNormal(seed=None))(hidden_layer1)
    hidden_layer2 = layers.BatchNormalization()(hidden_layer2)
    hidden_layer3 = layers.Dense(50, activation="selu", kernel_initializer=keras.initializers.LecunNormal(seed=None))(hidden_layer2)
    hidden_layer3 = layers.BatchNormalization()(hidden_layer3)
    rsna_output = layers.Dense(3, activation="softmax")(hidden_layer3)
    model = tf.keras.Model(rsna_input, rsna_output)

    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("keras_effnet_{0}_{1}.keras".format(condition_for_training, vertebrae_position))
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

    model.compile(loss="sparse_categorical_crossentropy", optimizer="adamax", metrics=["accuracy"])

I0000 00:00:1727504171.856667      13 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
with tpu_strategy.scope():
    history = model.fit(train_ds, class_weight={0:2, 1:1, 2:4}, epochs=7, validation_data=valid_ds
                        , callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/7


2024-09-28 06:19:51.150471: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node StatefulPartitionedCall.
I0000 00:00:1727504392.057317     840 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(f126606dc486d6b8:0:0), session_name()


  4/689 ━━━━━━━━━━━━━━━━━━━━ 12s 18ms/step - accuracy: 0.9388 - loss: 0.4622   

I0000 00:00:1727504409.508687     840 tpu_compile_op_common.cc:245] Compilation of f126606dc486d6b8:0:0 with session name  took 17.451326141s and succeeded
I0000 00:00:1727504409.577881     840 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(f126606dc486d6b8:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_6473348430844852907", property.function_library_fingerprint = 13904576599489135420, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1727504409.577940     840 tpu_compilation_cache_interface.cc:541] After adding entry for key f126606dc486d6b8:0

687/689 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.9627 - loss: 0.2935

I0000 00:00:1727504440.006501     835 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(b48def31d20c9421:0:0), session_name()
I0000 00:00:1727504443.841619     835 tpu_compile_op_common.cc:245] Compilation of b48def31d20c9421:0:0 with session name  took 3.835072264s and succeeded
I0000 00:00:1727504443.856279     835 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(b48def31d20c9421:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_8690839742036917094", property.function_library_fingerprint = 9047313440813943844, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "", property.guaranteed_constants_size = 0, embedding_partitions_fin

689/689 ━━━━━━━━━━━━━━━━━━━━ 64s 56ms/step - accuracy: 0.9627 - loss: 0.2935 - val_accuracy: 0.9636 - val_loss: 0.1828
Epoch 2/7
689/689 ━━━━━━━━━━━━━━━━━━━━ 34s 49ms/step - accuracy: 0.9627 - loss: 0.2821 - val_accuracy: 0.9636 - val_loss: 0.1729
Epoch 3/7
475/689 ━━━━━━━━━━━━━━━━━━━━ 8s 39ms/step - accuracy: 0.9618 - loss: 0.2840